<a href="https://colab.research.google.com/github/vedant-singh-007/DeepFake-Detection-Using-ResNext-50-LSTM/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import pandas as pd
from glob import glob


In [19]:
import os

# Adjust the path based on actual folder location
fake_folder = '/content/drive/MyDrive/fake_vids'
real_folder = '/content/drive/MyDrive/real_vids'

# Check if paths exist
print("Fake folder exists:", os.path.exists(fake_folder))
print("Real folder exists:", os.path.exists(real_folder))


Fake folder exists: True
Real folder exists: True


In [20]:
from glob import glob

fake_videos = sorted(glob(os.path.join(fake_folder, '*.mp4')))
real_videos = sorted(glob(os.path.join(real_folder, '*.mp4')))

print("Number of fake videos:", len(fake_videos))
print("Number of real videos:", len(real_videos))


Number of fake videos: 53
Number of real videos: 53


In [21]:
all_videos = fake_videos + real_videos
labels = [1]*len(fake_videos) + [0]*len(real_videos)  # 1 = Fake, 0 = Real

# Example:
for video, label in zip(all_videos[:5], labels[:5]):
    print(f"{video} -> {'Fake' if label == 1 else 'Real'}")


/content/drive/MyDrive/fake_vids/Copy of vs1.mp4 -> Fake
/content/drive/MyDrive/fake_vids/Copy of vs10.mp4 -> Fake
/content/drive/MyDrive/fake_vids/Copy of vs11.mp4 -> Fake
/content/drive/MyDrive/fake_vids/Copy of vs12.mp4 -> Fake
/content/drive/MyDrive/fake_vids/Copy of vs13.mp4 -> Fake


In [22]:
frame_count = []
for video_file in all_videos:
  cap = cv2.VideoCapture(video_file)
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames are " , frame_count)
print("Total no of video: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames are  [82, 110, 82, 101, 81, 85, 64, 79, 100, 102, 96, 100, 96, 78, 175, 100, 98, 102, 119, 102, 85, 127, 120, 74, 171, 114, 118, 92, 100, 92, 97, 150, 90, 94, 98, 104, 102, 102, 96, 102, 100, 92, 96, 79, 116, 99, 116, 104, 110, 81, 94, 79, 115, 81, 109, 81, 101, 80, 84, 64, 78, 100, 101, 95, 100, 95, 77, 174, 100, 98, 101, 118, 101, 84, 125, 120, 74, 171, 114, 118, 92, 100, 92, 96, 150, 90, 94, 98, 103, 101, 101, 95, 101, 100, 92, 96, 77, 116, 98, 116, 103, 109, 80, 94, 78, 115]
Total no of video:  106
Average frame per video: 100.86792452830188


In [6]:
output_video_folder = "/content/drive/MyDrive/cropped_face_videos"
os.makedirs(output_video_folder, exist_ok=True)

label_file_path = os.path.join(output_video_folder, "labels.csv")
label_entries = []


In [23]:
import cv2

def process_video_to_output(video_path, output_folder, label, max_frames=100, size=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []

    video_name = os.path.splitext(os.path.basename(video_path))[0]

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

        if len(faces) > 0:
            x, y, w, h = sorted(faces, key=lambda f: f[2]*f[3], reverse=True)[0]
            face_crop = frame[y:y+h, x:x+w]
            face_crop = cv2.resize(face_crop, size)
            frames.append(face_crop)

    cap.release()

    if len(frames) == 0:
        print(f"Skipping: {video_name} (no face detected)")
        return None

    if len(frames) < max_frames:
        last_frame = frames[-1]
        frames += [last_frame] * (max_frames - len(frames))
    else:
        frames = frames[:max_frames]

    # Write output video
    output_path = os.path.join(output_folder, f"{video_name}.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 25.0, size)

    for frame in frames:
        out.write(frame)
    out.release()

    return video_name, label


In [8]:
from tqdm import tqdm
import cv2 # Ensure cv2 is imported if not already in this cell
import os # Ensure os is imported if not already in this cell
from glob import glob # Ensure glob is imported if not already in this cell

# Download the Haar Cascade file if you don't have it
# You might need to run this only once
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml -P /content/

# Initialize the face cascade classifier
face_cascade = cv2.CascadeClassifier('/content/haarcascade_frontalface_default.xml')

# Check if the cascade file loaded correctly
if face_cascade.empty():
    print("Error loading cascade classifier!")

fake_folder = '/content/drive/MyDrive/fake_vids'
real_folder = '/content/drive/MyDrive/real_vids'

fake_videos = sorted(glob(os.path.join(fake_folder, '*.mp4')))
real_videos = sorted(glob(os.path.join(real_folder, '*.mp4')))

all_video_paths = [(path, 1) for path in fake_videos] + [(path, 0) for path in real_videos]

# Ensure the process_video_to_output function definition from the previous cell is run
# before running the loop below.

for path, label in tqdm(all_video_paths):
    result = process_video_to_output(path, output_video_folder, label)
    if result:
        video_name, label = result
        label_entries.append((video_name, label))

# You may want to save label_entries to a CSV file after the loop finishes
# df_labels = pd.DataFrame(label_entries, columns=['video_name', 'label'])
# df_labels.to_csv(label_file_path, index=False)

--2025-05-19 18:34:39--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 930127 (908K) [text/plain]
Saving to: ‘/content/haarcascade_frontalface_default.xml’

haarcascade_frontal 100%[===================>] 908.33K  --.-KB/s    in 0.03s   

2025-05-19 18:34:39 (33.5 MB/s) - ‘/content/haarcascade_frontalface_default.xml’ saved [930127/930127]



100%|██████████| 106/106 [22:24<00:00, 12.69s/it]


In [24]:

df_labels = pd.DataFrame(label_entries, columns=['video_name', 'label'])
df_labels.to_csv(label_file_path, index=False)

In [25]:
df_labels

,video_name,label
0,Copy of vs1,1
1,Copy of vs10,1
2,Copy of vs11,1
3,Copy of vs12,1
4,Copy of vs13,1
...,...,...
101,Copy of v53,0
102,Copy of v6,0
103,Copy of v7,0
104,Copy of v8,0


In [26]:
X=df_labels['video_name']
y=df_labels['label']

In [27]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [16]:
type(X_train)

pandas.core.series.Series

In [29]:
X_train.size

84

In [32]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt


In [33]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 9.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=d1dc4310706cdb4726d26e8366c6b39c3b698cdc99eec5c15e8c8561e046d952
  Stored in directory: /root/.cache/pip/wheels/04/52/ec/9355da79c29f160b038a20c784db2803c2f9fa2c8a462c176a
Successfully built face-recognition-models


In [37]:
import pandas as pd
import os

# Load CSV
df = pd.read_csv("/content/drive/MyDrive/cropped_face_videos/labels.csv")

# Convert labels: 'FAKE' -> 0, 'REAL' -> 1
df['label'] = df['label'].apply(lambda x: 0 if x == 'FAKE' else 1)

# Create dictionary: key = filename without extension, value = label
label_dict = dict(zip(df['video_name'].apply(lambda x: os.path.splitext(x)[0]), df['label']))

print(label_dict)


{'Copy of vs1': 1, 'Copy of vs10': 1, 'Copy of vs11': 1, 'Copy of vs12': 1, 'Copy of vs13': 1, 'Copy of vs14': 1, 'Copy of vs15': 1, 'Copy of vs16': 1, 'Copy of vs17': 1, 'Copy of vs18': 1, 'Copy of vs19': 1, 'Copy of vs2': 1, 'Copy of vs20': 1, 'Copy of vs21': 1, 'Copy of vs22': 1, 'Copy of vs23': 1, 'Copy of vs24': 1, 'Copy of vs25': 1, 'Copy of vs26': 1, 'Copy of vs27': 1, 'Copy of vs28': 1, 'Copy of vs29': 1, 'Copy of vs3': 1, 'Copy of vs30': 1, 'Copy of vs31': 1, 'Copy of vs32': 1, 'Copy of vs33': 1, 'Copy of vs34': 1, 'Copy of vs35': 1, 'Copy of vs36': 1, 'Copy of vs37': 1, 'Copy of vs38': 1, 'Copy of vs39': 1, 'Copy of vs4': 1, 'Copy of vs40': 1, 'Copy of vs41': 1, 'Copy of vs42': 1, 'Copy of vs43': 1, 'Copy of vs44': 1, 'Copy of vs45': 1, 'Copy of vs46': 1, 'Copy of vs47': 1, 'Copy of vs48': 1, 'Copy of vs49': 1, 'Copy of vs5': 1, 'Copy of vs50': 1, 'Copy of vs51': 1, 'Copy of vs52': 1, 'Copy of vs53': 1, 'Copy of vs6': 1, 'Copy of vs7': 1, 'Copy of vs8': 1, 'Copy of vs9': 1, '

In [39]:
class VideoDataSet(Dataset):
  def __init__(self, video_paths,label_dict,sequence_length=100, transform=None):
    self.video_paths = video_paths
    self.label_dict = label_dict
    self.transform = transform
    self.seq_length=sequence_length

  def __len__():
    return len(self.video_paths)
  def __getitem__(self, index):
      path = self.video_paths[index]
      video_name = os.path.splitext(os.path.basename(path))[0]
      label = self.label_dict.get(video_name, -1)  # default -1 if not found
      frames = self.extract_frames(path)   #its an array of 224x224x3 images
      video_tensor = torch.tensor(np.stack(frames), dtype=torch.float32)  # shape: (seq_len, H, W, C)
      return video_tensor, label
  def extract_frames(self, path):
        cap = cv2.VideoCapture(path)
        frames = []

        while len(frames) < self.seq_length:
            success, frame = cap.read()
            if not success:
                break
            frame = cv2.resize(frame, self.frame_size)
            frames.append(frame)

        cap.release()
        return frames



